# Egocentric Video Mapper

Before starting make sure you have an available GPU. If you are unsure about whether you have an available GPU or if you want to check which GPU you will be working with run the next cell.

In [ ]:
!nvidia-smi -L

This next cell will import the modules and functions that will be used for this Alpha Lab

In [9]:
from pathlib import Path

from gaze_mapper import EgocentricMapper
from optic_flow import calculate_optic_flow
from sync_videos import OffsetCalculator
from utils import (
    generate_comparison_video_kwargs,
    generate_mapper_kwargs,
    write_timestamp_csv,
)
from video_handler import VideoHandler
from video_renderer import save_comparison_video, save_gaze_video

Fill out the path to the directory of the uncompressed `Timeseries Data + Scene Video` download from Cloud.

In [3]:
neon_timeseries_dir = "Path/To/NeonTimeSeriesFolder"

Fill out the path of the corresponding alternative egocentric view recording (please make sure that both videos have the same orientation).

In [4]:
alternative_vid_path = "Path/To/AlternativeVideo.ext"

Define an output directory where the different output files will be saved.

In [5]:
output_dir = "Path/To/OutputFolder"

## Step 1: Video Synchronization

In this first step, optic flow in both videos will be calculated. Each optic flow result is saved to a csv file in an automatically created optic_flow subdirectory in the output directory. The csv file will contain the following columns:
- start: The start frame timestamp of the optic flow calculation
- end: The end frame timestamp of the optic flow calculation
- dx: Average horizontal displacement  
- dy: Average vertical displacement 
- angle: Average angular change in degrees

Optic flow is then used to obtain the time offset of the alternative egocentric video with respect to the Neon Scene video. 

There are two available algorithms that can be used: Lukas-Kanade and Farneback. Select one of them in the next cell.



In [6]:
optic_flow_algorithm = "Lucas-Kanade"  # options: "Lucas-Kanade", "Farneback"

In [ ]:
neon_of_result, alternative_of_result = calculate_optic_flow(
    neon_timeseries_dir, alternative_vid_path, output_dir, optic_flow_algorithm
)

Once optic flow is measured, the time offset will be calculated and an alternative_camera_timestamps.csv will be generated with a similar format as Neon's world_timestamps.csv.

In [ ]:
offset_calc = OffsetCalculator(
    src=alternative_of_result["dy"].values,
    src_timestamps=alternative_of_result["start"].values,
    dst=neon_of_result["dy"].values,
    dst_timestamps=neon_of_result["start"].values,
    resampling_frequency=500,
)

t_offset, pearson_corr = offset_calc.estimate_time_offset()
print(
    f"Estimated offset of alternative egocentric video with respect to Neon scene video: {t_offset} seconds (Pearson correlation: {pearson_corr})"
)

write_timestamp_csv(
    neon_timeseries_dir,
    VideoHandler(alternative_vid_path).timestamps + t_offset,
    output_file_dir=output_dir,
)

## Step 2: Map gaze data to the action video

After synchronizing both videos, it is time to obtain Neon gaze signal in the coordinate system of the other video. 


<div class="alert alert-block alert-info">
Here we make use of deep learning methods to guide the gaze mapping. By default, this notebook uses an implementation of Efficient LOFTR. We have implemented the following algorithms: 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'. If you wish to try them out, just write the name of the desired algorithm in the <b>image_matcher_choice</b> variable found in the cell below.
</div>

In [7]:
image_matcher_choice = "Efficient_LOFTR"  # Options: 'Efficient_LOFTR', 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'

mapper_kwargs = generate_mapper_kwargs(
    neon_timeseries_dir,
    alternative_vid_path,
    output_dir,
    image_matcher_choice,
)

mapper = EgocentricMapper(**mapper_kwargs)

Before running the mapping pipeline you can adjust how often (in seconds) you want the image correspondances refreshed. 


- The higher you set the time threshold the less time the mapping will take, however it may decrease the accuracy of the mapping. 
- The lower the time threshold the more time the mapping will take. 

<div class="alert alert-block alert-warning">

If you leave the value at 'None' then for every gaze new correspondences will be obtained. 
We recommend using values smaller than 0.5 seconds.
</div>
 

In [8]:
time_threshold = 0.5

In [ ]:
mapped_gaze_path = mapper.map_gaze(refresh_time_thrshld=time_threshold)
print(f"Gaze mapped to alternative camera saved at {mapped_gaze_path}")

## Step 3 (OPTIONAL): Create visualization videos

For visualization purposes you can get a side-to-side rendering with gaze of the Neon Scene video and the alternative egocentric video.  


In [ ]:
comparison_video_kwargs = generate_comparison_video_kwargs(
    neon_timeseries_dir,
    alternative_vid_path,
    mapped_gaze_path,
    output_dir,
)
save_comparison_video(**comparison_video_kwargs)

Additionally, you can get only the alternative egocentric video (at its original frame rate) with the overlaid gaze circle.

In [ ]:
gaze_video_args = {
    "video_path": alternative_vid_path,
    "timestamps_path": Path(output_dir, "alternative_camera_timestamps.csv"),
    "gaze_path": Path(mapped_gaze_path),
    "save_video_path": Path(
        output_dir,
        "alternative_camera_gaze_overlay.mp4",
    ),
}

save_gaze_video(**gaze_video_args)